In [1]:
import os
import shutil
import pandas as pd
import pyreadstat
import numpy as np
from PIL import Image
import pydicom
import matplotlib.pyplot as plt
import cv2

In [2]:
data, meta = pyreadstat.read_sav('Dr.miratashi_3 copy.sav')

In [3]:
def move_folders(folder_list, source_directory, destination_directory):
    for folder_name in folder_list:
        source_path = os.path.join(source_directory, folder_name)
        destination_path = os.path.join(destination_directory, folder_name)
        shutil.move(source_path, destination_path)
        print(f"Moved folder '{folder_name}' from '{source_directory}' to '{destination_directory}'")

In [4]:
def create_pid_folder(parent_directory, pid1):
    for folder_name in pid1:
        if type(folder_name) == str:
            folder_path = os.path.join(parent_directory, folder_name)
        else:
            folder_path = os.path.join(parent_directory, str(folder_name))
            
        os.makedirs(folder_path, exist_ok=True)

        # Create the "FGT" and "BPE" sub folders within each folder
        fgt_folder_path = os.path.join(folder_path, 'FGT')
        bpe_folder_path = os.path.join(folder_path, 'BPE')
        os.makedirs(fgt_folder_path, exist_ok=True)
        os.makedirs(bpe_folder_path, exist_ok=True)

In [5]:
def automated_copy(origin_directory, destination_directory, pid2, fgt_idx, bpe_idx):
    present_data = []
    present_data = get_folder_names(destination_directory, present_data)
    create_pid_folder(destination_directory, pid2)
    unique_idx = []
    img_folder_names = []
    for idx3, patient_id in enumerate(pid2):
        if str(patient_id) in present_data:
            continue
        else:
            unique_idx.append(idx3)
            dcm_directory = origin_directory + str(patient_id) + '/IMAGE/DCM/'
            for k, (root, dirs, files) in enumerate(os.walk(dcm_directory)):
                if k == 0:
                    img_folder_names.append(dcm_directory + str(dirs[k]))

    c = 0           
    for idx1, patient_id in enumerate(pid2):
        print(f"i={idx1} | pid={patient_id}")
        if idx1 not in unique_idx:
            print(f"{idx1} is not in the unique idx")
            continue
        else:
            start_fgt, end_fgt = fgt_idx[idx1]
            start_bpe, end_bpe = bpe_idx[idx1]
            
            difference_fgt = end_fgt - start_fgt
            difference_bpe = end_bpe - start_bpe
            if difference_fgt != difference_bpe:
                print("Slice Number is Different for FGT and BPE")
                break
            destination_fgt_folder = destination_directory + str(patient_id) + '/FGT'
            destination_bpe_folder = destination_directory + str(patient_id) + '/BPE'
            for j1 in range(start_fgt, end_fgt+1):
                if len(str(j1)) == 4:
                    fgt_img_directory = img_folder_names[c] + '/I000' + str(j1)
                else:
                    fgt_img_directory = img_folder_names[c] + '/I0000' + str(j1)
                shutil.copy(fgt_img_directory, destination_fgt_folder)
                
            for j2 in range(start_bpe, end_bpe+1):
                if len(str(j2)) == 4:
                    bpe_img_directory = img_folder_names[c] + '/I000' + str(j2)
                else:
                    bpe_img_directory = img_folder_names[c] + '/I0000' + str(j2)
                shutil.copy(bpe_img_directory, destination_bpe_folder)
            c += 1

In [6]:
def get_folder_names(path, folders):
    for entry in os.scandir(path):
        if entry.is_dir():
            folders.append(entry.name)
    return folders

In [7]:
def get_file_names(files_path):
    files_names = os.listdir(files_path)
    return files_names

In [8]:
def normalize_array(array):
    # Normalize the array to the range [0, 255]
    min_val = np.min(array)
    max_val = np.max(array)
    normalized_array = (array - min_val) * (255.0 / (max_val - min_val))

    # Convert the array to uint8 data type
    normalized_array = normalized_array.astype(np.uint8)

    return normalized_array

In [9]:
def slice_checker(fgt_list, bpe_list, pid_list):
    for i in range(len(fgt_list)):
        start_fgt, end_fgt = fgt_list[i]
        start_bpe, end_bpe = bpe_list[i]
        
        difference_fgt = end_fgt - start_fgt
        difference_bpe = end_bpe - start_bpe
        if difference_fgt != difference_bpe:
            print(f"Slice Number is Different for FGT and BPE of {pid_list[i]}")

In [10]:
def img_preprocess(dicom_img_path, t_percentage, margin_value, output_shape=None):
    ds = pydicom.dcmread(dicom_img_path)
    pixel_array = ds.pixel_array
    
    img_max = np.max(pixel_array)
    t = np.round(t_percentage/100 * img_max)
    pixel_array[pixel_array<t] = 0
    
    pixel_array = normalize_array(pixel_array)
    pixel_array = np.rot90(pixel_array, k=2)
    
    img_row, img_column = pixel_array.shape
    
    # Find P1
    p1 = [0, 0]
    flag = False
    for row in range(img_row):
        if flag:
            break
            
        for column in range(img_column//2):
            if pixel_array[row, column] != 0:
                p1 = [row, column]
                flag = True
                break
            
    # Find P2
    p2 = [0, 0]
    flag = False
    for row in range(img_row):
        if flag:
            break
    
        for column in range(img_column-1, img_column//2, -1):
            if pixel_array[row, column] != 0:
                p2 = [row, column]
                flag = True
                break
            
    # Find P3
    p3_column = (p1[1] + p2[1])//2
    p3 = [0, 0]
    for row in range(165, img_row):
        if pixel_array[row, p3_column] != 0:
            p3 = [row, p3_column]
            break
            
    img_upper = pixel_array[1:(p3[0]+margin_value),:]
    img_lower = pixel_array[(p3[0]+1):img_row,:]

    final_image = np.zeros((img_row, img_column), dtype=np.uint8)
    final_image[1:(p3[0]+margin_value), :] = img_upper
    
    if output_shape != None:
        final_image = cv2.resize(src=final_image, dsize=output_shape, interpolation=cv2.INTER_CUBIC)
    
    return final_image, p3

In [11]:
def bpe_process(dicom_img_path, t_percentage, margin_value, p3, output_shape=None):
    ds = pydicom.dcmread(dicom_img_path)
    pixel_array = ds.pixel_array
    
    # img_max = np.max(pixel_array)
    # t = np.round(t_percentage/100 * img_max)
    # pixel_array[pixel_array<t] = 0
    
    pixel_array = normalize_array(pixel_array)
    pixel_array = np.rot90(pixel_array, k=2)
    
    img_row, img_column = pixel_array.shape
    
    img_upper = pixel_array[1:(p3[0]+margin_value),:]
    img_lower = pixel_array[(p3[0]+1):img_row,:]

    final_image = np.zeros((img_row, img_column), dtype=np.uint8)
    final_image[1:(p3[0]+margin_value), :] = img_upper
    
    if output_shape != None:
        final_image = cv2.resize(src=final_image, dsize=output_shape, interpolation=cv2.INTER_CUBIC)
        
    return final_image

In [12]:
def save_array_as_image(array, file_path, file_format='PNG'):
    # Convert the NumPy array to PIL image
    image = Image.fromarray(array)
    # Save the image file
    image.save(file_path, format=file_format)

In [13]:
def dicom_to_png(files_path, destination_path, slice_number=None, out_shape=None):
    final_folder_names = []
    final_folder_names = get_folder_names(files_path, final_folder_names)  # Folder names have str data type
    final_folder_names.sort()
    
    create_pid_folder(destination_path, final_folder_names)
    
    for p_idx, pid in enumerate(final_folder_names):
        print(f"index: {p_idx}  |  pid: {pid}")
        fgt_path = files_path + pid + '/FGT/'
        bpe_path = files_path + pid + '/BPE/'
        
        fgt_dicom_names = get_file_names(fgt_path)  # File names have str data type
        bpe_dicom_names = get_file_names(bpe_path)  # File names have str data type
        fgt_dicom_names.sort()
        bpe_dicom_names.sort()
        
        center_index = len(fgt_dicom_names)//2
        
        if slice_number == None:
            index_range = np.arange(center_index-center_index, center_index+center_index)
        else:
            index_range = np.arange(center_index-slice_number//2, center_index+slice_number//2)
            
        for idx, data_index in enumerate(index_range):
            fgt_file_path = fgt_path + fgt_dicom_names[data_index]
            fgt_destination_path = destination_path + pid + '/FGT/' + str(idx) + '.png'
            img, p3 = img_preprocess(dicom_img_path=fgt_file_path, t_percentage=10, margin_value=15, output_shape=out_shape)
            save_array_as_image(img, fgt_destination_path, file_format='PNG')
            
            bpe_file_path = bpe_path + bpe_dicom_names[data_index]
            bpe_destination_path = destination_path + pid + '/BPE/' + str(idx) + '.png'
            img = bpe_process(dicom_img_path=bpe_file_path, t_percentage=10, margin_value=15, p3=p3, output_shape=out_shape)
            save_array_as_image(img, bpe_destination_path, file_format='PNG')

# Data Preprocessing
## Step 1
### At this step, we get the unique patient IDs and remove data with multiple diagnosis.

In [44]:
folder_names = []

mpath = 'E:/Breast Data/Breast/'
folder_names = get_folder_names(mpath, folder_names)

mpath = 'E:/Breast Data/breast newNAC'
folder_names = get_folder_names(mpath, folder_names)

mpath = 'H:/Breast MRI/01/sps'
folder_names = get_folder_names(mpath, folder_names)

mpath = 'H:/Breast MRI/02/sps'
folder_names = get_folder_names(mpath, folder_names)

mpath = 'H:/Breast MRI/03/breast new'
folder_names = get_folder_names(mpath, folder_names)

mpath = 'H:/Breast MRI/03/breast new 2'
folder_names = get_folder_names(mpath, folder_names)

mpath = 'H:/Breast MRI/03/nac'
folder_names = get_folder_names(mpath, folder_names)

mpath = 'H:/Breast MRI/03/sps'
folder_names = get_folder_names(mpath, folder_names)

mpath = 'G:/Breast Data/nac'
folder_names = get_folder_names(mpath, folder_names)

mpath = 'G:/Breast Data/sps'
folder_names = get_folder_names(mpath, folder_names)


In [381]:
folder_names = np.array(folder_names)

In [382]:
folder_names_unique = np.unique(folder_names)

In [383]:
data_codes = np.array(data.code)
data_codes_unique = np.unique(data_codes)

In [384]:
same_codes_unique = []
for i in folder_names_unique:
    if i in data_codes_unique:
        same_codes_unique.append(i)
        
same_codes_unique = np.array(same_codes_unique)

In [385]:
not_in_dataset = []
for i in data_codes_unique:
    if i not in same_codes_unique:
        not_in_dataset.append(i)

not_in_dataset_unique = np.unique(np.array(not_in_dataset))

In [386]:
same_codes_unique_mdd_removed = []
for i in same_codes_unique:
    if np.array(data[data.code == i].FG).shape[0] > 1:
        fg_i = np.array(data[data.code == i].FG)
        bpe_i = np.array(data[data.code == i].BPE)
        if np.all(fg_i==fg_i[0]) and np.all(bpe_i==bpe_i[0]):
            same_codes_unique_mdd_removed.append(i)
    else:
        same_codes_unique_mdd_removed.append(i)

In [387]:
not_in_dataset_unique_mdd_removed = []
for i in not_in_dataset_unique:
    if np.array(data[data.code == i].FG).shape[0] > 1:
        fg_i = np.array(data[data.code == i].FG)
        bpe_i = np.array(data[data.code == i].BPE)
        if np.all(fg_i==fg_i[0]) and np.all(bpe_i==bpe_i[0]):
            not_in_dataset_unique_mdd_removed.append(i)
    else:
        not_in_dataset_unique_mdd_removed.append(i)
        

In [388]:
all_ok_data = same_codes_unique_mdd_removed + not_in_dataset_unique_mdd_removed

In [389]:
len(all_ok_data)

1146

# Part 1

In [390]:
mpath = 'H:/Breast MRI/01/sps'
folder_names_part1 = []
folder_names_part1 = get_folder_names(mpath, folder_names_part1)

In [391]:
for name in folder_names_part1:
    if name not in all_ok_data:
        print(name)

In [392]:
no_data_cases = ['9222704', '9999465', '8881092', '11258955', '11843612']

In [393]:
len(all_ok_data)

1146

In [394]:
for pid in no_data_cases:
    if pid in all_ok_data:
        all_ok_data.remove(pid)

In [395]:
len(all_ok_data)

1142

In [45]:
# Origin and destination directory
d_directory = 'E:/Selected Data/'
o_directory = 'H:/Breast MRI/01/sps/'

In [46]:
patients_ids_1 = [8860408, 8861603, 8861971, 8866708, 8889836, 8915593, 8915632, 8926218, 8934757, 8935156, 8939476, 8977117,
                  9008081, 9017845, 9108321, 9162757, 9170060, 9173422, 9174037, 9182263, 9221819, 9239385, 9246035, 9354593,
                  9362401, 9363679, 9410575, 9459081, 9459302, 9459663, 9471658, 9471667, 9471756, 9471997, 9474758, 9484852,
                  9484938, 9486031, 9486352, 9492065, 9512135, 9535197, 9535278, 9535434, 9559948, 9571865, 9577964, 9578077,
                  9638156, 9638749, 9646672, 9647040, 9649691, 9658124, 9661700, 9670311, 9670327, 9673120, 9673179, 9673778,
                  9674633, 9680775, 9680867, 9700791, 9701051, 9718836, 9726839, 9727663, 9739752, 9757971, 9773985, 9852342,
                  9900364, 9906687, 9906752, 9910509, 9918129, 9918268, 9918324, 9920466, 9924862, 9926215, 9929361, 9929533,
                  9930243, 9953872, 9953955, 9954110, 9966610, 9970531, 9971081, 9975377, 9975397, 9985427, 9991646, 9994490,
                  9997801, 9997958, 9998057, 10006533, 10007023, 10007991, 10010860, 10018165, 10030277, 10032605, 10032779,
                  10045260, 11201493, 11201533, 11205832, 11208303, 11211696, 11218656, 11224788, 11231020, 11237684, 11239822,
                  11244816, 11247503, 11254075, 11256378, 11264398, 11272908, 11273475, 11286927, 11290544, 11299936, 11301332,
                  11306141, 11331505, 11333648, 11333741, 11338126, 11338192, 11342837, 11344516, 11348359, 11348891, 11351690,
                  11352729, 11355812, 11356851, 11358818, 11366628, 11366836, 11367575, 11369417, 11369686, 11371493, 11372703,
                  11375497, 11377087, 11379700, 11384586, 11385160, 11387992, 11388246, 11389044, 11390408, 11391724, 11393372,
                  11396654, 11572209, 11573971, 11582084, 11582317, 11585576, 11589250, 11840108, 11842798, 11842875, 11846812,
                  11850448, 11851264, 11863836, 11879754, 11881596, 11884718, 11885867, 11887918, 11894552, 11899869, 11900471,
                  11900937, 11903746, 11913796, 11913939, 11915334, 11917076, 11917317]

In [47]:
fgt_1 = [(300, 451), (373, 458), (558, 685), (396, 481), (536, 621), (379, 550), (434, 517), (335, 478), (582, 661), (587, 662),
         (303, 454), (348, 523), (533, 618), (365, 440), (514, 599), (335, 510), (405, 548), (589, 674), (505, 590), (348, 499),
         (521, 606), (387, 542), (407, 492), (554, 639), (365, 450), (365, 450), (432, 513), (490, 567), (371, 442), (464, 549),
         (365, 524), (365, 528), (422, 573), (338, 485), (591, 676), (493, 668), (393, 560), (432, 587), (595, 758), (325, 448),
         (365, 528), (365, 438), (302, 367), (365, 430), (365, 438), (501, 586), (569, 654), (529, 614), (277, 356), (277, 352),
         (281, 366), (249, 314), (490, 575), (420, 505), (277, 352), (536, 679), (277, 352), (421, 506), (474, 559), (439, 524), 
         (441, 526), (471, 560), (315, 406), (449, 536), (421, 510), (466, 551), (332, 499), (269, 354), (433, 518), (479, 650), 
         (461, 546), (435, 520), (552, 735), (431, 518), (443, 530), (423, 586), (402, 545), (485, 628), (380, 523), (430, 515), 
         (496, 671), (597, 800), (385, 464), (451, 540), (299, 384), (313, 404), (315, 402), (317, 404), (435, 520), (448, 639), 
         (612, 771), (439, 526), (517, 602), (500, 585), (405, 490), (555, 640), (426, 513), (297, 382), (277, 350), (506, 649), 
         (464, 549), (454, 539), (455, 598), (471, 556), (435, 598), (407, 554), (426, 577), (299, 384), (467, 662), (421, 612), 
         (376, 551), (258, 343), (275, 450), (429, 620), (337, 512), (500, 675), (350, 525), (396, 575), (406, 581), (405, 490), 
         (380, 555), (413, 604), (393, 568), (417, 592), (383, 558), (423, 508), (404, 579), (417, 640), (461, 604), (402, 577), 
         (286, 461), (359, 534), (587, 762), (399, 574), (385, 560), (420, 607), (370, 545), (417, 502), (405, 490), (395, 570), 
         (552, 727), (363, 538), (567, 718), (369, 544), (407, 582), (415, 602), (393, 568), (426, 601), (385, 560), (389, 564), 
         (344, 519), (355, 530), (428, 619), (561, 756), (397, 572), (670, 857), (402, 577), (356, 519), (412, 591), (366, 541),
         (366, 541), (343, 518), (383, 554), (394, 585), (414, 593), (412, 587), (423, 618), (453, 628), (359, 534), (451, 626),
         (390, 565), (439, 614), (385, 576), (312, 475), (497, 672), (434, 625), (410, 585), (946, 1145), (904, 1095), (868, 1059),
         (347, 522), (314, 489), (387, 562), (395, 570), (348, 523), (396, 587), (380, 559), (588, 803), (301, 460), (593, 772),
         (384, 559)]

In [48]:
bpe_1 = [(1060, 1211), (920, 1005), (2589, 2716), (914, 999), (1162, 1247), (1615, 1786), (1042, 1125), (1235, 1378), (1054, 1133),
         (1035, 1110), (1243, 1394), (1408, 1583), (1071, 1156), (813, 888), (1022, 1107), (1395, 1570), (1349, 1492), (1157, 1242),
         (1061, 1146), (1288, 1439), (1047, 1132), (1323, 1478), (925, 1010), (1080, 1165), (886, 971), (915, 1000), (928, 1009),
         (978, 1055), (820, 891), (980, 1065), (1233, 1392), (1253, 1416), (1252, 1403), (1146, 1293), (1141, 1226), (1441, 1616),
         (1301, 1468), (1280, 1435), (1699, 1862), (1225, 1348), (1253, 1416), (803, 876), (700, 765), (763, 828), (803, 876),
         (1007, 1092), (1107, 1192), (1063, 1148), (769, 848), (749, 824), (803, 888), (671, 736), (1032, 1117), (954, 1039),
         (749, 824), (2267, 2410), (749, 824), (959, 1044), (1012, 1097), (977, 1062), (979, 1064), (1025, 1114), (889, 980), 
         (1002, 1089), (978, 1067), (1015, 1100), (1238, 1405), (783, 868), (1203, 1288), (2382, 2553), (1126, 1211), (969, 1054), 
         (1872, 2055), (984, 1071), (990, 1077), (1423, 1586), (1278, 1421), (1373, 1516), (1256, 1399), (1228, 1313), (1768, 1943), 
         (2001, 2204), (869, 948), (1011, 1100), (849, 934), (880, 971), (868, 955), (871, 958), (957, 1042), (1816, 2007), 
         (2760, 2919), (971, 1058), (1150, 1235), (1155, 1240), (927, 1012), (1227, 1312), (1065, 1152), (834, 919), (754, 827), 
         (1226, 1369), (1161, 1246), (1107, 1192), (1487, 1630), (1108, 1193), (1411, 1574), (1303, 1450), (1342, 1493), (845, 930), 
         (1859, 2054), (1813, 2004), (1648, 1823), (792, 877), (1511, 1686), (1833, 2024), (1579, 1754), (1736, 1911), (1610, 1785), 
         (1716, 1895), (1646, 1821), (936, 1021), (1616, 1791), (1793, 1984), (1635, 1810), (1725, 1900), (1643, 1818), (957, 1042), 
         (1820, 1995), (2723, 2946), (1337, 1480), (1746, 1921), (1522, 1697), (1667, 1842), (1859, 2034), (1671, 1846), (1657, 1832), 
         (1788, 1975), (1610, 1785), (948, 1033), (939, 1024), (1631, 1806), (1788, 1963), (1683, 1858), (1327, 1478), (1605, 1780), 
         (1651, 1826), (1731, 1918), (1635, 1810), (1914, 2089), (1657, 1832), (1673, 1848), (1580, 1755), (1651, 1826), (1820, 2011),
         (1977, 2172), (1653, 1828), (2640, 2827), (1638, 1813), (1520, 1683), (1708, 1887), (1602, 1777), (1602, 1777), (1603, 1778),
         (1595, 1766), (1762, 1953), (1746, 1925), (1684, 1859), (1803, 1998), (1689, 1864), (1667, 1842), (1687, 1862), (1630, 1805),
         (1681, 1856), (1753, 1944), (1464, 1627), (1733, 1908), (1814, 2005), (1652, 1827), (2580, 2779), (2464, 2655), (2408, 2599),
         (1583, 1758), (1370, 1545), (1631, 1806), (1639, 1814), (1590, 1765), (1752, 1943), (2290, 2469), (2842, 3057), (1441, 1600),
         (1861, 2040), (1626, 1801)]

In [49]:
# slice_checker(fgt_1, bpe_1, patients_ids_1)

In [50]:
# automated_copy(o_directory, d_directory, patients_ids_1, fgt_1, bpe_1)

i=0 | pid=8860408
i=1 | pid=8861603
i=2 | pid=8861971
i=3 | pid=8866708
i=4 | pid=8889836
i=5 | pid=8915593
i=6 | pid=8915632
i=7 | pid=8926218
i=8 | pid=8934757
i=9 | pid=8935156
i=10 | pid=8939476
i=11 | pid=8977117
i=12 | pid=9008081
i=13 | pid=9017845
i=14 | pid=9108321
i=15 | pid=9162757
i=16 | pid=9170060
i=17 | pid=9173422
i=18 | pid=9174037
i=19 | pid=9182263
i=20 | pid=9221819
i=21 | pid=9239385
i=22 | pid=9246035
i=23 | pid=9354593
i=24 | pid=9362401
i=25 | pid=9363679
i=26 | pid=9410575
i=27 | pid=9459081
i=28 | pid=9459302
i=29 | pid=9459663
i=30 | pid=9471658
i=31 | pid=9471667
i=32 | pid=9471756
i=33 | pid=9471997
i=34 | pid=9474758
i=35 | pid=9484852
i=36 | pid=9484938
i=37 | pid=9486031
i=38 | pid=9486352
i=39 | pid=9492065
i=40 | pid=9512135
i=41 | pid=9535197
i=42 | pid=9535278
i=43 | pid=9535434
i=44 | pid=9559948
i=45 | pid=9571865
i=46 | pid=9577964
i=47 | pid=9578077
i=48 | pid=9638156
i=49 | pid=9638749
i=50 | pid=9646672
i=51 | pid=9647040
i=52 | pid=9649691
i=5

# Part 2

In [830]:
mpath = 'H:/Breast MRI/02/sps'
folder_names_part2 = []
folder_names_part2 = get_folder_names(mpath, folder_names_part2)

In [403]:
for name in folder_names_part2:
    if name not in all_ok_data:
        print(name)

In [404]:
no_data_cases = ['1191379', '8368349', '8832712', '8145891', '8302127', '8305735', '8414121', '8675439', '8675610', '8682515']

In [405]:
len(all_ok_data)

1142

In [406]:
for pid in no_data_cases:
    if pid in all_ok_data:
        all_ok_data.remove(pid)

In [407]:
len(all_ok_data)

1135

In [51]:
# Origin and destination directory
d_directory = 'E:/Selected Data/'
o_directory = 'H:/Breast MRI/02/sps/'

In [52]:
patients_ids_2 = [969995, 1568016, 8647484, 8668594, 8668972, 8838792, 8844951, 8845106, 8859611]

In [53]:
fgt_2 = [(413, 498), (405, 490), (315, 478), (374, 549), (348, 523), (382, 533), (385, 560), (303, 466), (481, 566)]

In [54]:
bpe_2 = [(953, 1038), (957, 1042), (1135, 1298), (1434, 1609), (1408, 1583), (1322, 1473), (1445, 1620), (1303, 1466), (1123, 1208)]

In [55]:
# slice_checker(fgt_2, bpe_2, patients_ids_2)

In [56]:
# automated_copy(o_directory, d_directory, patients_ids_2, fgt_2, bpe_2)

i=0 | pid=969995
i=1 | pid=1568016
i=2 | pid=8647484
i=3 | pid=8668594
i=4 | pid=8668972
i=5 | pid=8838792
i=6 | pid=8844951
i=7 | pid=8845106
i=8 | pid=8859611


# Part 3

In [813]:
mpath = 'H:/Breast MRI/03/breast new'
folder_names_part3 = []
folder_names_part3 = get_folder_names(mpath, folder_names_part3)

In [415]:
for name in folder_names_part3:
    if name not in all_ok_data:
        print(name)

In [416]:
no_data_cases = []

In [417]:
len(all_ok_data)

1135

In [418]:
for pid in no_data_cases:
    if pid in all_ok_data:
        all_ok_data.remove(pid)

In [419]:
len(all_ok_data)

1135

In [57]:
# Origin and destination directory
d_directory = 'E:/Selected Data/'
o_directory = 'H:/Breast MRI/03/breast new/'

In [58]:
patients_ids_3 = [9947622, 9971081, 10006533, 10500639, 10506817, 10507162, 10509355, 10520523, 10532430, 10540595,
                  10553401, 10554186, 10561171, 10581017, 10591458, 10632327, 10716690, 10718503, 10723940, 10738155,
                  10743894, 10756274, 10773065, 10791568, 10812027]

In [59]:
fgt_3 = [(333, 500), (612, 771), (506, 649), (881, 1072), (982, 1173), (1287, 1478), (1017, 1208), (1190, 1389), (1033, 1224), (991, 1182), 
         (1052, 1243), (1160, 1351), (978, 1169), (879, 1038), (1027, 1218), (1224, 1415), (974, 1153), (1694, 1885), (498, 641), (1134, 1325), 
         (1155, 1346), (1174, 1365), (1174, 1365), (1250, 1441), (1181, 1372)]

In [60]:
bpe_3 = [(1999, 2166), (2760, 2919), (1226, 1369), (2225, 2416), (2356, 2547), (3133, 3324), (2391, 2582), (3269, 3468), (2577, 2768), 
         (2941, 3132), (2558, 2749), (2688, 2879), (2928, 3119), (2687, 2846), (2571, 2762), (2776, 2967), (2430, 2609), (3250, 3441), 
         (2396, 2539), (2688, 2879), (2567, 2758), (2718, 2909), (2729, 2920), (2824, 3015), (2785, 2976)]

In [61]:
# slice_checker(fgt_3, bpe_3, patients_ids_3)

In [62]:
# automated_copy(o_directory, d_directory, patients_ids_3, fgt_3, bpe_3)

i=0 | pid=9947622
i=1 | pid=9971081
1 is not in the unique idx
i=2 | pid=10006533
2 is not in the unique idx
i=3 | pid=10500639
i=4 | pid=10506817
i=5 | pid=10507162
i=6 | pid=10509355
i=7 | pid=10520523
i=8 | pid=10532430
i=9 | pid=10540595
i=10 | pid=10553401
i=11 | pid=10554186
i=12 | pid=10561171
i=13 | pid=10581017
i=14 | pid=10591458
i=15 | pid=10632327
i=16 | pid=10716690
i=17 | pid=10718503
i=18 | pid=10723940
i=19 | pid=10738155
i=20 | pid=10743894
i=21 | pid=10756274
i=22 | pid=10773065
i=23 | pid=10791568
i=24 | pid=10812027


# Part 4

In [426]:
mpath = 'H:/Breast MRI/03/breast new 2'
folder_names_part4 = []
folder_names_part4 = get_folder_names(mpath, folder_names_part4)

In [427]:
folder_list = []
for name in folder_names_part4:
    if name not in all_ok_data:
        print(name)
        folder_list.append(name)

In [428]:
len(folder_list)

0

In [429]:
# source_directory = mpath
# destination_directory = "H:/Breast MRI/03/NotOkData/breast new 2"  # Replace with the actual destination directory
# move_folders(folder_list, source_directory, destination_directory)

In [430]:
no_data_cases = []

In [431]:
len(all_ok_data)

1135

In [432]:
for pid in no_data_cases:
    if pid in all_ok_data:
        all_ok_data.remove(pid)

In [433]:
len(all_ok_data)

1135

In [63]:
# Origin and destination directory
d_directory = 'E:/Selected Data/'
o_directory = 'H:/Breast MRI/03/breast new 2/'

In [64]:
patients_ids_4 = [10942269, 10983337, 11032357, 11034705, 11075626, 11076778, 11086355, 11180494, 11183489, 11216685, 11275661, 
                  11346010, 11405138, 11597250, 11610585, 11618270, 11627699, 11757932, 11758205, 11785907, 11789411, 11800353,
                  11801650, 11821274, 11863126, 11863650, 11879076, 11881596, 11884718, 11885867, 11913084, 12047098, 12086263]

In [65]:
fgt_4 = [(1263, 1454), (1141, 1332), (2281, 2484), (1865, 2068), (867, 1006), (1204, 1395), (1246, 1437), (1177, 1380), (998, 1189),
         (1178, 1389), (916, 1107), (1397, 1576), (1320, 1503), (861, 1012), (790, 965), (1229, 1444), (896, 1087), (769, 940),
         (755, 906), (841, 1004), (979, 1170), (1226, 1417), (1362, 1553), (1054, 1245), (514, 669), (700, 867), (1050, 1241),
         (946, 1145), (904, 1095), (868, 1059), (400, 547), (1124, 1303), (592, 787)]

In [66]:
bpe_4 = [(2813, 3004), (3200, 3391), (4025, 4228), (3648, 3851), (2033, 2172), (2768, 2959), (2796, 2987), (2810, 3013), (2542, 2733),
         (2881, 3092), (2460, 2651), (2849, 3028), (2810, 2993), (2133, 2284), (3467, 3642), (2967, 3182), (2440, 2631), (2173, 2344),
         (2021, 2172), (2219, 2382), (2556, 2747), (2770, 2961), (2900, 3091), (2628, 2819), (2168, 2323), (2490, 2657),
         (3962, 4153), (2580, 2779), (2464, 2655), (2408, 2599), (1946, 2093), (2598, 2777), (2638, 2833)]

In [67]:
# slice_checker(fgt_4, bpe_4, patients_ids_4)

In [68]:
# automated_copy(o_directory, d_directory, patients_ids_4, fgt_4, bpe_4)

i=0 | pid=10942269
i=1 | pid=10983337
i=2 | pid=11032357
i=3 | pid=11034705
i=4 | pid=11075626
i=5 | pid=11076778
i=6 | pid=11086355
i=7 | pid=11180494
i=8 | pid=11183489
i=9 | pid=11216685
i=10 | pid=11275661
i=11 | pid=11346010
i=12 | pid=11405138
i=13 | pid=11597250
i=14 | pid=11610585
i=15 | pid=11618270
i=16 | pid=11627699
i=17 | pid=11757932
i=18 | pid=11758205
i=19 | pid=11785907
i=20 | pid=11789411
i=21 | pid=11800353
i=22 | pid=11801650
i=23 | pid=11821274
i=24 | pid=11863126
i=25 | pid=11863650
i=26 | pid=11879076
i=27 | pid=11881596
27 is not in the unique idx
i=28 | pid=11884718
28 is not in the unique idx
i=29 | pid=11885867
29 is not in the unique idx
i=30 | pid=11913084
i=31 | pid=12047098
i=32 | pid=12086263


# Part 5

In [440]:
mpath = 'H:/Breast MRI/03/nac'
folder_names_part5 = []
folder_names_part5 = get_folder_names(mpath, folder_names_part5)

In [441]:
folder_list = []
for name in folder_names_part5:
    if name not in all_ok_data:
        print(name)
        folder_list.append(name)

In [442]:
len(folder_list)

0

In [443]:
# source_directory = mpath
# destination_directory = "H:/Breast MRI/03/NotOkData/nac"  # Replace with the actual destination directory
# move_folders(folder_list, source_directory, destination_directory)

In [444]:
no_data_cases = []

In [445]:
len(all_ok_data)

1135

In [446]:
for pid in no_data_cases:
    if pid in all_ok_data:
        all_ok_data.remove(pid)

In [447]:
len(all_ok_data)

1135

In [69]:
# Origin and destination directory
d_directory = 'E:/Selected Data/'
o_directory = 'H:/Breast MRI/03/nac/'

In [70]:
patients_ids_5 = [10500639, 10506817, 10507162, 10509355, 10520523, 10532430, 10540595, 10553401, 10554186, 10561171, 10581017, 10591458, 
                10632327, 10716690, 10718503, 10738155, 10743894, 10756274, 10773065, 10791568, 10795621, 10812027, 10879476, 10905568, 
                10916400, 10937977, 10959273, 10983337, 11032357, 11034705, 11075626, 11076778, 11086355, 11180494, 11183489, 11216685,
                11275661, 11346010, 11627699]

In [71]:
fgt_5 = [(941,1132), (982,1173), (1287,1478), (1017,1208), (1190,1389), (1033,1224),
 (991,1182), (1052,1243), (1160,1351), (978,1169),(879,1038), (1027,1218), (1224,1415), (974,1153), (1694,1885),
 (1134,1325), (1155,1346), (1174,1365), (1174,1365), (1250,1441), (405,490), (1181,1372), (1038,1229), (1007,1194),
 (631,806), (963,1154), (536,679), (1141,1332), (2281,2484), (1865,2068), (867,1006), (1204,1395), (1246,1437),
 (1177,1380), (998,1189), (1178,1389), (916,1107), (1397,1576), (896,1087)
 ]

In [72]:
bpe_5 = [(2285,2476), (2416,2607), (3199,3390), (2451,2642), (3269,3468), (2577,2768), (2941,3132), (2558,2749), (2688,2879), (2928,3119), (2687,2846), 
(2571,2762), (2776,2967), (2430,2609), (3250,3441), (2688,2879),(2567,2758), (2718,2909), (2729,2920), (2824,3015), (957,1042), (2785,2976), (2582,2773),
(2523,2710), (1915,2090), (2507,2698), (2116,2259), (3200,3391), (4025,4228), (3648,3851), (2033,2172), (2768,2959), (2796,2987), (2810,3013), (2542,2733),
(2881,3092), (2460,2651), (2849,3028), (2440,2631)
]

In [73]:
# slice_checker(fgt_5, bpe_5, patients_ids_5)

In [74]:
# automated_copy(o_directory, d_directory, patients_ids_5, fgt_5, bpe_5)

i=0 | pid=10500639
0 is not in the unique idx
i=1 | pid=10506817
1 is not in the unique idx
i=2 | pid=10507162
2 is not in the unique idx
i=3 | pid=10509355
3 is not in the unique idx
i=4 | pid=10520523
4 is not in the unique idx
i=5 | pid=10532430
5 is not in the unique idx
i=6 | pid=10540595
6 is not in the unique idx
i=7 | pid=10553401
7 is not in the unique idx
i=8 | pid=10554186
8 is not in the unique idx
i=9 | pid=10561171
9 is not in the unique idx
i=10 | pid=10581017
10 is not in the unique idx
i=11 | pid=10591458
11 is not in the unique idx
i=12 | pid=10632327
12 is not in the unique idx
i=13 | pid=10716690
13 is not in the unique idx
i=14 | pid=10718503
14 is not in the unique idx
i=15 | pid=10738155
15 is not in the unique idx
i=16 | pid=10743894
16 is not in the unique idx
i=17 | pid=10756274
17 is not in the unique idx
i=18 | pid=10773065
18 is not in the unique idx
i=19 | pid=10791568
19 is not in the unique idx
i=20 | pid=10795621
i=21 | pid=10812027
21 is not in the uni

# Part 6

In [454]:
mpath = 'H:/Breast MRI/03/sps'
folder_names_part6 = []
folder_names_part6 = get_folder_names(mpath, folder_names_part6)

In [455]:
folder_list = []
for name in folder_names_part6:
    if name not in all_ok_data:
        print(name)
        folder_list.append(name)

In [456]:
len(folder_list)

0

In [457]:
# source_directory = mpath
# destination_directory = "H:/Breast MRI/03/NotOkData/sps"  # Replace with the actual destination directory
# move_folders(folder_list, source_directory, destination_directory)

In [458]:
no_data_cases = ['8390792', '8564813', '8613792', '8756913']

In [459]:
len(all_ok_data)

1135

In [460]:
for pid in no_data_cases:
    if pid in all_ok_data:
        all_ok_data.remove(pid)

In [461]:
len(all_ok_data)

1131

In [75]:
# Origin and destination directory
d_directory = 'E:/Selected Data/'
o_directory = 'H:/Breast MRI/03/sps/'

In [76]:
patients_ids_6 = [8308544, 8320097, 8644677, 8654723, 8668965, 8693340, 8845788, 8846009, 8943630, 8980696, 8987973, 8994596, 8995280,
                  9002280, 9010579, 9010854, 9011738, 9012992, 9013052, 9017038, 9021496, 9024061, 9024191, 9024288, 9024861, 9064702,
                  9066182, 9072270, 9075718, 9075921, 9082292, 9099448, 9099484, 9099535, 9102847, 9103559, 9103635, 9124636, 9132962,
                  9135746, 9137960, 9178082, 9186604, 9187014, 9211330, 9221344, 9246145, 9250562, 9250859, 9251259, 9257467, 9260607,
                  9278242, 9278286, 9278507, 9280993, 9282859, 9297184, 9300041, 9303450, 9303958, 9306355, 9314842, 9314854,
                  9329696, 9329801, 9332914, 9338697, 9338721, 9350416, 9354498, 9374330, 9374338, 9374628, 9376635, 9378242, 9380740, 9385156,
                  9385880, 9397693, 9397802, 9398002, 9400054, 9401113, 9403123, 9407013, 9408592, 9409404, 9433277, 9433866, 9434346,
                  9446950, 9447103, 9449885, 9452045, 9456472, 9456991, 9555999, 9571196, 9590052, 9613321, 9613957, 9634850, 9657991,
                  9670407, 9679589, 9683733, 9692694, 9692981, 9730700, 9749506]

In [77]:
fgt_6 = [(364,515), (658,833), (356,413), (321,472), (536,679), (536,623), (348,523), (345,512), (356,531), (518,615), (348,523),
         (387,472), (396,481), (338,501), (605,690), (622,707), (340,423), (431,516), (365,450), (387,472), (269,354), (365,450),
         (509,594), (365,450), (505,590), (454,625), (335,502), (348,523), (549,634), (525,610), (545,688), (509,594), (378,463),
         (482,653), (625,710), (521,606), (580,665), (529,614), (320,471), (430,561), (344,479), (427,512), (365,442), (378,453),
         (350,517), (509,594), (421,506), (394,479), (397,482), (403,488), (661,746), (409,494), (332,475), (397,540), (371,522),
         (411,562), (568,653), (335,498), (421,506), (393,536), (368,511), (809,894), (393,478), (467,556), (551,694), (365,430),
         (439,524), (450,535), (365,450), (451,536), (537,622), (427,512), (365,450), (330,481), (549,634), (529,614), (375,550),
         (365,430), (387,474), (399,542), (413,572), (391,460), (541,626), (597,682), (521,606), (373,458), (365,432), (401,486),
         (517,602), (521,606), (633,718), (377,528), (348,503), (554,639), (574,659), (417,502), (411,496), (509,594), (489,574),
         (396,481), (403,472), (275,350), (301,394), (405,470), (277,352), (273,348), (415,504), (459,544), (505,590), (519,604),
         (273,344)]

In [78]:
bpe_6 = [(1124,1275), (1606,1781), (782,839), (1081,1232), (2092,2235), (1056,1143), (1408,1583), (1245,1412), (1416,1591),
         (1206,1303), (1408,1583), (885,970), (894,979), (1338,1501), (1103,1188), (1124,1209), (848,931), (1317,1402), (1262,1347),
         (971,1056), (699,784), (1247,1332), (1458,1543), (1126,1211), (1015,1100), (1554,1725), (1523,1690), (1408,1583), (1150,1235),
         (1081,1166), (1601,1744), (1151,1236), (876,961), (1410,1581), (1205,1290), (1083,1168), (1198,1283), (1105,1190), (1236,1387),
         (1402,1533), (1340,1475), (1102,1187), (823,900), (826,901), (1370,1537), (1023,1108), (939,1024), (1123,1208), (1202,1287),
         (1244,1329), (1211,1296), (951,1036), (1208,1351), (1273,1416), (1287,1438), (1375,1526), (1106,1191), (1335,1498), (1101,1186),
         (1269,1412), (1244,1387), (1515,1600), (904,989), (1228,1317), (1427,1570), (763,828), (981,1066), (948,1033), (1101,1186),
         (1209,1294), (1079,1164), (925,1010), (863,948), (1270,1421), (1111,1196), (1063,1148), (1435,1610), (1089,1154), (908,995),
         (1275,1418), (1369,1528), (809,878), (1087,1172), (1135,1220), (1047,1132), (907,992), (1126,1193), (914,999), (1039,1124),
         (1047,1132), (1179,1264), (1293,1444), (1308,1463), (1080,1165), (1120,1205), (943,1028), (933,1018), (1242,1327), (1256,1341),
         (937,1022), (845,914), (747,822), (863,956), (842,907), (749,824), (745,820), (969,1058), (1136,1221), (1170,1255), (1073,1158),
         (725,796)]

In [79]:
# slice_checker(fgt_6, bpe_6, patients_ids_6)

In [80]:
# automated_copy(o_directory, d_directory, patients_ids_6, fgt_6, bpe_6)

i=0 | pid=8308544
i=1 | pid=8320097
i=2 | pid=8644677
i=3 | pid=8654723
i=4 | pid=8668965
i=5 | pid=8693340
i=6 | pid=8845788
i=7 | pid=8846009
i=8 | pid=8943630
i=9 | pid=8980696
i=10 | pid=8987973
i=11 | pid=8994596
i=12 | pid=8995280
i=13 | pid=9002280
i=14 | pid=9010579
i=15 | pid=9010854
i=16 | pid=9011738
i=17 | pid=9012992
i=18 | pid=9013052
i=19 | pid=9017038
i=20 | pid=9021496
i=21 | pid=9024061
i=22 | pid=9024191
i=23 | pid=9024288
i=24 | pid=9024861
i=25 | pid=9064702
i=26 | pid=9066182
i=27 | pid=9072270
i=28 | pid=9075718
i=29 | pid=9075921
i=30 | pid=9082292
i=31 | pid=9099448
i=32 | pid=9099484
i=33 | pid=9099535
i=34 | pid=9102847
i=35 | pid=9103559
i=36 | pid=9103635
i=37 | pid=9124636
i=38 | pid=9132962
i=39 | pid=9135746
i=40 | pid=9137960
i=41 | pid=9178082
i=42 | pid=9186604
i=43 | pid=9187014
i=44 | pid=9211330
i=45 | pid=9221344
i=46 | pid=9246145
i=47 | pid=9250562
i=48 | pid=9250859
i=49 | pid=9251259
i=50 | pid=9257467
i=51 | pid=9260607
i=52 | pid=9278242
i=5

# Part 7

In [468]:
mpath = "E:/Breast Data/Breast"
folder_names_part7 = []
folder_names_part7 = get_folder_names(mpath, folder_names_part7)

In [469]:
folder_list = []
for name in folder_names_part7:
    if name not in all_ok_data:
        print(name)
        folder_list.append(name)

In [470]:
len(folder_list)

0

In [471]:
# source_directory = mpath
# destination_directory = "E:/Breast Data/NotOkData/Breast"  # Replace with the actual destination directory
# move_folders(folder_list, source_directory, destination_directory)

In [472]:
no_data_cases = []

In [473]:
len(all_ok_data)

1131

In [474]:
for pid in no_data_cases:
    if pid in all_ok_data:
        all_ok_data.remove(pid)

In [475]:
len(all_ok_data)

1131

In [81]:
# Origin and destination directory
d_directory = 'E:/Selected Data/'
o_directory = 'E:/Breast Data/Breast/'

In [82]:
patients_ids_7 = [10879476, 10905568, 10937977, 10983337, 11032357, 11034705, 11075626, 11076778, 11086355, 11180494, 11183489,
11216685, 11275661, 11346010, 11405138, 11597250, 11610585, 11618270, 11627699, 11748892, 11757932, 11758205, 11785907, 11789411, 11800353, 11801650, 11863126, 11863650, 11879076]

In [83]:
fgt_7 = [(1038,1229), (1007,1194), (963,1154), (1141,1332), (2281,2484), (1865,2068), (867,1006), (1204,1395), (1246,1437), (1177,1380), (998,1189), 
(1178,1389), (916,1107), (1397,1576), (1258,1441), (805,956), (736,911), (1155,1370), (832,1023), (812,1003), (721,892), (707,858), (783,946), (907,1098), (1146,1337), (1292,1483), (514,669), (700,867), (970,1161)]

In [84]:
bpe_7 = [(2582,2773), (2523,2710), (2507,2698), (3200,3391), (4025,4228), (3648,3851), (2033,2172), (2768,2959), (2796,2987), (2810,3013), (2542,2733),
(2881,3092), (2460,2651), (2849,3028), (2748,2931), (2077,2228), (3413,3588), (2893,3108), (2376,2567), (2360,2551), (2125,2296), (1973,2124), (2161,2324), (2484,2675), (2690,2881), (2830,3021), (2168,2323), (2490,2657), (3882,4073) ]

In [85]:
# slice_checker(fgt_7, bpe_7, patients_ids_7)

In [86]:
# automated_copy(o_directory, d_directory, patients_ids_7, fgt_7, bpe_7)

i=0 | pid=10879476
0 is not in the unique idx
i=1 | pid=10905568
1 is not in the unique idx
i=2 | pid=10937977
2 is not in the unique idx
i=3 | pid=10983337
3 is not in the unique idx
i=4 | pid=11032357
4 is not in the unique idx
i=5 | pid=11034705
5 is not in the unique idx
i=6 | pid=11075626
6 is not in the unique idx
i=7 | pid=11076778
7 is not in the unique idx
i=8 | pid=11086355
8 is not in the unique idx
i=9 | pid=11180494
9 is not in the unique idx
i=10 | pid=11183489
10 is not in the unique idx
i=11 | pid=11216685
11 is not in the unique idx
i=12 | pid=11275661
12 is not in the unique idx
i=13 | pid=11346010
13 is not in the unique idx
i=14 | pid=11405138
14 is not in the unique idx
i=15 | pid=11597250
15 is not in the unique idx
i=16 | pid=11610585
16 is not in the unique idx
i=17 | pid=11618270
17 is not in the unique idx
i=18 | pid=11627699
18 is not in the unique idx
i=19 | pid=11748892
i=20 | pid=11757932
20 is not in the unique idx
i=21 | pid=11758205
21 is not in the uni

# Part 8

In [482]:
mpath = "E:/Breast Data/breast newNAC"
folder_names_part8 = []
folder_names_part8 = get_folder_names(mpath, folder_names_part8)

In [483]:
folder_list = []
for name in folder_names_part8:
    if name not in all_ok_data:
        print(name)
        folder_list.append(name)

In [484]:
len(folder_list)

0

In [485]:
# source_directory = mpath
# destination_directory = "E:/Breast Data/NotOkData/breast newNAC"  # Replace with the actual destination directory
# move_folders(folder_list, source_directory, destination_directory)

In [486]:
no_data_cases = []

In [487]:
len(all_ok_data)

1131

In [488]:
for pid in no_data_cases:
    if pid in all_ok_data:
        all_ok_data.remove(pid)

In [489]:
len(all_ok_data)

1131

In [87]:
# Origin and destination directory
d_directory = 'E:/Selected Data/'
o_directory = 'E:/Breast Data/breast newNAC/'

In [88]:
patients_ids_8 = [8308544, 8331255, 8356670, 8378220, 8379234, 8380175, 8380177, 8397654, 8420266, 8476139, 8502426, 8647484, 8647633, 8830555, 8860408, 8866532, 10812027, 10879476, 11748892, 11913796, 11913939]

In [89]:
fgt_8 = [(364,515), (280,411), (300,451), (288,439), (304,467), (284,435), (290,421), (316,479), (300,451), (258,377), (329,500), (315,478), (348,531), (300,451), (300,451), (321,488), (1181,1372), (1038,1229), (876,1067), (380,559), (588,803)]

In [90]:
bpe_8 = [(1124,1275), (940,1071), (1880,2031), (1081,1232), (1176,1339), (1088,1239), (950,1081), (1136,1299), (1060,1211), (858,977), (1189,1360), (1135,1298), (1268,1451), (1060,1211), (1060,1211), (1413,1580), (2785,2976), (2582,2773), (2424,2615), (2290,2469), (2842,3057) ]

In [91]:
# slice_checker(fgt_8, bpe_8, patients_ids_8)

In [92]:
# automated_copy(o_directory, d_directory, patients_ids_8, fgt_8, bpe_8)

i=0 | pid=8308544
0 is not in the unique idx
i=1 | pid=8331255
i=2 | pid=8356670
i=3 | pid=8378220
i=4 | pid=8379234
i=5 | pid=8380175
i=6 | pid=8380177
i=7 | pid=8397654
i=8 | pid=8420266
i=9 | pid=8476139
i=10 | pid=8502426
i=11 | pid=8647484
11 is not in the unique idx
i=12 | pid=8647633
i=13 | pid=8830555
i=14 | pid=8860408
14 is not in the unique idx
i=15 | pid=8866532
i=16 | pid=10812027
16 is not in the unique idx
i=17 | pid=10879476
17 is not in the unique idx
i=18 | pid=11748892
18 is not in the unique idx
i=19 | pid=11913796
19 is not in the unique idx
i=20 | pid=11913939
20 is not in the unique idx


# Part 9

In [496]:
mpath = "G:/Breast Data/nac"
folder_names_part9 = []
folder_names_part9 = get_folder_names(mpath, folder_names_part9)

In [497]:
folder_list = []
for name in folder_names_part9:
    if name not in all_ok_data:
        print(name)
        folder_list.append(name)

In [498]:
len(folder_list)

0

In [499]:
# source_directory = mpath
# destination_directory = "G:/Breast Data/NotOkData/nac"  # Replace with the actual destination directory
# move_folders(folder_list, source_directory, destination_directory)

In [500]:
no_data_cases = []

In [501]:
len(all_ok_data)

1131

In [502]:
for pid in no_data_cases:
    if pid in all_ok_data:
        all_ok_data.remove(pid)

In [503]:
len(all_ok_data)

1131

In [93]:
# Origin and destination directory
d_directory = 'E:/Selected Data/'
o_directory = 'G:/Breast Data/nac'

In [94]:
patients_ids_9 = [11405138, 11597250, 11610585, 11618270, 11748892, 11757932, 11758205, 11785907, 11789411, 11800353, 11801650, 11879076, 11881596, 11885867]

In [95]:
fgt_9 = [(1019,1210), (861,1012), (790,965), (1229,1444), (876,1067), (769,940), (755,906), (841,1004), (979,1170), (1226,1417), (1362,1553), (1050,1241), (946,1145), (868,1059)]

In [96]:
bpe_9 = [(2567,2758), (2133,2284), (3467,3642), (2967,3182), (2424,2615), (2173,2344), (2021,2172), (2219,2382), (2556,2747), (2770,2961), (2900,3091), (3962,4153), (2580,2779), (2408,2599)]

In [97]:
# slice_checker(fgt_9, bpe_9, patients_ids_9)

In [98]:
# automated_copy(o_directory, d_directory, patients_ids_9, fgt_9, bpe_9)

i=0 | pid=11405138
0 is not in the unique idx
i=1 | pid=11597250
1 is not in the unique idx
i=2 | pid=11610585
2 is not in the unique idx
i=3 | pid=11618270
3 is not in the unique idx
i=4 | pid=11748892
4 is not in the unique idx
i=5 | pid=11757932
5 is not in the unique idx
i=6 | pid=11758205
6 is not in the unique idx
i=7 | pid=11785907
7 is not in the unique idx
i=8 | pid=11789411
8 is not in the unique idx
i=9 | pid=11800353
9 is not in the unique idx
i=10 | pid=11801650
10 is not in the unique idx
i=11 | pid=11879076
11 is not in the unique idx
i=12 | pid=11881596
12 is not in the unique idx
i=13 | pid=11885867
13 is not in the unique idx


# Part 10

In [510]:
mpath = "G:/Breast Data/sps"
folder_names_part10 = []
folder_names_part10 = get_folder_names(mpath, folder_names_part10)

In [511]:
folder_list = []
for name in folder_names_part10:
    if name not in all_ok_data:
        print(name)
        folder_list.append(name)

In [512]:
len(folder_list)

0

In [513]:
# source_directory = mpath
# destination_directory = "G:/Breast Data/NotOkData/sps"  # Replace with the actual destination directory
# move_folders(folder_list, source_directory, destination_directory)

In [514]:
no_data_cases = ['8867592']

In [515]:
len(all_ok_data)

1131

In [516]:
for pid in no_data_cases:
    if pid in all_ok_data:
        all_ok_data.remove(pid)

In [517]:
len(all_ok_data)

1130

In [99]:
# Origin and destination directory
d_directory = 'E:/Selected Data/'
o_directory = 'G:/Breast Data/sps/'

In [100]:
patients_ids_10 = [8343185, 8409056, 8416350, 8479601, 8534099, 8866532, 8873492, 8889724, 8915531, 8925092, 9014143, 9158896, 9290074, 9290715, 9306747, 9494072, 9495256, 9507637, 9511809, 9511838, 9513877, 9514578, 9516448, 9521942, 9522082, 9522435,
9522620, 9821838, 9824657, 9833318, 9833973, 9834400, 9839404, 9839939, 9840299, 9845434, 9847483, 9847488, 9852399, 9961155, 10013259, 10014191, 10022835, 10023779, 10029875, 10029952, 10030148, 10532430, 10540595, 10553401, 10561171, 10609248,
11195971, 11216300, 11227862, 11227939, 11251756, 11256225, 11265500, 11267809, 11269597, 11279181, 11279333, 11279953, 11282806, 11290557, 11295079, 11295090, 11297877, 11297940, 11299279, 11306332, 11309011, 11312299, 11312329, 11316461, 11316477,
11319133, 11323537, 11331203, 11331360, 11334261, 11344443, 11348952, 11351904, 11355417, 11361965
]

In [101]:
fgt_10 = [(368,511), (400,567), (753,920), (424,567), (442,585), (321,488), (486,561), (485,570), (425,604), (372,455), (417,494), (641,720), (396,483), (654,739), (525,610), (538,623), (552,637), (453,652), (365,532), (408,473), (521,606), (558,643), (550,635), (434,519), (499,560),  (493,564),
(529,614), (277,362), (471,572), (405,490), (381,444), (405,490), (293,378), (310,387), (275,352), (396,481), (277,362), (277,362), (405,490), (419,504), (405,482), (465,554), (311,396), (405,490), (361,504), (430,593), (344,475), (639,782), (991,1182), (1052,1243), (978,1169), (304,415),
(406,593), (395,574), (347,522), (347,522), (275,450), (427,618), (316,491), (395,578), (282,457), (532,699), (428,615), (414,605), (503,654), (325,500), (336,511), (377,552), (402,585), (396,595), (677,872), (323,462), (333,508), (393,568), (355,530), (360,535), (325,476), 
(336,503), (461,636), (333,508), (404,579), (384,559), (383,558), (405,490), (386,561), (407,582), (516,703)
]

In [102]:
bpe_10 = [(1930,2073), (1300,1467), (1653,1820), (2130,2273), (2301,2444), (1413,1580), (950,1025), (1090,1175), (1786,1965), (916,999), (1322,1399), (1144,1223), (938,1025), (1355,1440), (1055,1140), (1068,1153), (1078,1163), (1893,2092), (1273,1440), (806,871), (1069,1154), (1088,1173), (1092,1177), (946,1031), (901,962), (945,1016),
(1448,1533), (799,884), (1101,1202), (951,1036), (813,876), (1082,1167), (1111,1196), (792,869), (757,834), (1078,1163), (799,884), (799,884), (927,1012), (977,1062), (887,964), (1035,1124), (854,939), (939,1024), (1237,1380), (1406,1569), (1160,1291), (2510,2653), (2941,3132), (2558,2749), (2928,3119), (864,975),
(2392,2579), (1679,1858), (1619,1794), (1619,1794), (1511,1686), (1831,2022), (1552,1727), (1739,1922), (1518,1693), (1804,1971), (2434,2621), (1818,2009), (1419,1570), (1561,1736), (1608,1783), (1637,1812), (1734,1917), (2473,2672), (2105,2300), (1307,1446), (1569,1744), (1737,1912), (1651,1826), (1596,1771), (1417,1568), 
(2082,2249), (1706,1881), (1569,1744), (1640,1815), (2093,2268), (1635,1810), (939,1024), (1802,1977), (1739,1914), (1860,2047) 
]

In [103]:
# slice_checker(fgt_10, bpe_10, patients_ids_10)

In [104]:
# automated_copy(o_directory, d_directory, patients_ids_10, fgt_10, bpe_10)

i=0 | pid=8343185
i=1 | pid=8409056
i=2 | pid=8416350
i=3 | pid=8479601
i=4 | pid=8534099
i=5 | pid=8866532
5 is not in the unique idx
i=6 | pid=8873492
i=7 | pid=8889724
i=8 | pid=8915531
i=9 | pid=8925092
i=10 | pid=9014143
i=11 | pid=9158896
i=12 | pid=9290074
i=13 | pid=9290715
i=14 | pid=9306747
i=15 | pid=9494072
i=16 | pid=9495256
i=17 | pid=9507637
i=18 | pid=9511809
i=19 | pid=9511838
i=20 | pid=9513877
i=21 | pid=9514578
i=22 | pid=9516448
i=23 | pid=9521942
i=24 | pid=9522082
i=25 | pid=9522435
i=26 | pid=9522620
i=27 | pid=9821838
i=28 | pid=9824657
i=29 | pid=9833318
i=30 | pid=9833973
i=31 | pid=9834400
i=32 | pid=9839404
i=33 | pid=9839939
i=34 | pid=9840299
i=35 | pid=9845434
i=36 | pid=9847483
i=37 | pid=9847488
i=38 | pid=9852399
i=39 | pid=9961155
i=40 | pid=10013259
i=41 | pid=10014191
i=42 | pid=10022835
i=43 | pid=10023779
i=44 | pid=10029875
i=45 | pid=10029952
i=46 | pid=10030148
i=47 | pid=10532430
47 is not in the unique idx
i=48 | pid=10540595
48 is not in th

# DICOM to PNG File

In [14]:
files_path = 'E:/Selected Data/'
destination_path = 'E:/Selected Data 128/'

In [16]:
dicom_to_png(files_path, destination_path, slice_number=2, out_shape=(128, 128))

index: 0  |  pid: 10006533
index: 1  |  pid: 10007023
index: 2  |  pid: 10007991
index: 3  |  pid: 10010860
index: 4  |  pid: 10013259
index: 5  |  pid: 10014191
index: 6  |  pid: 10018165
index: 7  |  pid: 10022835
index: 8  |  pid: 10023779
index: 9  |  pid: 10029875
index: 10  |  pid: 10029952
index: 11  |  pid: 10030148
index: 12  |  pid: 10030277
index: 13  |  pid: 10032605
index: 14  |  pid: 10032779
index: 15  |  pid: 10045260
index: 16  |  pid: 10500639
index: 17  |  pid: 10506817
index: 18  |  pid: 10507162
index: 19  |  pid: 10509355
index: 20  |  pid: 10520523
index: 21  |  pid: 10532430
index: 22  |  pid: 10540595
index: 23  |  pid: 10553401
index: 24  |  pid: 10554186
index: 25  |  pid: 10561171
index: 26  |  pid: 10581017
index: 27  |  pid: 10591458
index: 28  |  pid: 10609248
index: 29  |  pid: 10632327
index: 30  |  pid: 10716690
index: 31  |  pid: 10718503
index: 32  |  pid: 10723940
index: 33  |  pid: 10738155
index: 34  |  pid: 10743894
index: 35  |  pid: 10756274
in